<a href="https://colab.research.google.com/github/trahinhasan/Marketplace-Product-Title-Quality-Classification/blob/main/marketplace_product_title_quality_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/sample_data/walmart_products_free_dataset.csv")

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df = df[['title']]

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset=['title'])

In [ ]:
df = df.drop_duplicates(subset=["title"])

In [ ]:
df.sample(10)

In [ ]:
df.head()

In [ ]:
import re

def clean_title(text):
  text = text.lower()
  text = re.sub(r'[^a-z0-9 ]', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  return text.strip()


In [ ]:
df['clean_title'] = df['title'].apply(clean_title)

In [ ]:
brands = ['samsung','apple','nike','sony','lg','hp','dell']
colors = ['black','white','red','blue','green','silver']
materials = ['cotton','leather','plastic','steel','wood']



In [ ]:
def assign_label(title):
    tokens = title.split()
    length = len(tokens)

    brand_present = any(b in title for b in brands)
    color_present = any(c in title for c in colors)
    material_present = any(m in title for m in materials)
    number_present = any(char.isdigit() for char in title)

    score = sum([brand_present, color_present, material_present, number_present])

    if score >= 3 and length >= 5:
        return "Good"
    elif score >= 1 and length >= 3:
        return "Mediocre"
    else:
        return "Bad"


In [ ]:
df['label'] = df['clean_title'].apply(assign_label)


In [ ]:
df['label'].value_counts()


In [ ]:
def extract_features(title):
    return pd.Series({
        'title_length': len(title.split()),
        'brand_count': sum(b in title for b in brands),
        'color_count': sum(c in title for c in colors),
        'material_count': sum(m in title for m in materials),
        'number_count': sum(char.isdigit() for char in title)
    })


In [ ]:
features = df['clean_title'].apply(extract_features)


In [ ]:
X = features
y = df['label']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42
)

model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
import pandas as pd

importance = pd.Series(model.feature_importances_, index=X.columns)
importance.sort_values(ascending=False)


In [ ]:
import pickle

with open("title_quality_model.pkl", "wb") as f:
    pickle.dump(model, f)
